# Analysis Pipeline for Intracranial Data

<!-- ### Triggers/Events
Using photo triggers

### Preprocessing
TODO: ADD ouline of preprocessing steps for iEEG data

### MNE python
Working on the MNE library to recreate the same matlab funcitons: 
Location:
https://mne.tools/stable/auto_tutorials/intro/10_overview.html#sphx-glr-auto-tutorials-intro-10-overview-py -->


Let's go

## IMPORT AND LOAD DATA

In [2]:
# IMPORTS: 
from sklearn.preprocessing import StandardScaler
import os
import numpy as np
import mne
import scipy
from scipy import signal
from mne.utils import _TempDir
import matplotlib.pyplot as plt
%matplotlib qt

import pd_parser
from pd_parser.parse_pd import _read_raw, _to_tsv
out_dir = _TempDir()
print(f'Data location.... : {out_dir}')
# DATA LOCATION:
data_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/PAT_3066'
# data_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/S20'
script_path = '/Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/Scripts'


t_onset_s= 275
t_offset_s= 1620
# LOAD DATA 
# data_raw_file = os.path.join(data_path, 'S20_20181217_014956.mff')
# data_raw_file = os.path.join(data_path, 'PAT_3066_EEG_695997_Anon_FLM.edf')
data_raw_file = os.path.join(data_path, 'PAT_3066_EEG_708977_Anon_CategoryLocalizer.edf')
raw = mne.io.read_raw_edf(data_raw_file,exclude=['ECG+','ECG-','MKR4+','Xe2', 'Xe3', 'Xe4', 'Xe5', 'Xe6'])
print(raw.__len__)
picks = mne.pick_channels_regexp(raw.info.ch_names, regexp='Xe1')
# picks = mne.pick_channels_regexp(raw.info.ch_names, regexp='')
# raw.plot(order=picks,n_channels=203, event_color='cyan', scalings=3500, duration=200)
raw.get_data(picks=picks)
raw2 = raw[-1][0]
raw2 = np.array(raw2[0])

del raw

Data location.... : /var/folders/85/ct7qgj4500s62pw9zxkv_xc40000gn/T/tmp_mne_tempdir_hon8ju7x
Extracting EDF parameters from /Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/PAT_3066/PAT_3066_EEG_708977_Anon_CategoryLocalizer.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
<bound method BaseRaw.__len__ of <RawEDF | PAT_3066_EEG_708977_Anon_CategoryLocalizer.edf, 204 x 2181120 (1065.0 s), ~196 kB, data not loaded>>


## VISUALIZE TRIGGER DATA: PHOTODIODE

In [28]:
%matplotlib qt
print(np.shape(raw2))
import pyqtgraph as pg
import pyqtgraph as plotWidget
time_s = range(len(raw2))
pg.plot(time_s,raw2)


(2181120,)


## NORMALIZE TRIGGER DATA: PHOTODIODE

In [14]:
def normalized(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))

raw2 = normalized(raw2)
pg.plot(time_s,raw2)


## PREPROCESSING TRIGGER DATA TO GET EVENTS: PHOTODIODE

#### SAMPLE

In [4]:
## SMOOTH
from savitzky_golay import savitzky_golay
from get_trigger_indexes_photodiode import get_trigger_indexes_photodiode
# print(raw2)
# temp = savitzky_golay(raw2, 11, 3) # window size 51, polynomial order 3
index_onset, index_offset = get_trigger_indexes_photodiode()
print(len(index_onset), len(index_offset))
# time2 = time_s[10000:50000]
# temp2 = temp[0:40000]
# temp3 = np.diff(temp[0:40000])
# plotWidget = pg.plot(title="Trigger raw + diff: sample")

# plotWidget.plot(time2,temp2,pen='blue')
# plotWidget.plot(time2[0:-1],temp3,pen='white')


*** USING EXAMPLE FILE ***
*   Data location.... : /var/folders/85/ct7qgj4500s62pw9zxkv_xc40000gn/T/tmp_mne_tempdir_andabsy_


Extracting EDF parameters from /Volumes/GoogleDrive/My Drive/EEG_DATA_PIERRE/PAT_3066/PAT_3066_EEG_708977_Anon_CategoryLocalizer.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
*    length of onset indexes
*   length of offset indexes
113 110


#### FULL SIZE

In [16]:
time2 = time_s
temp2 = temp
temp3 = np.diff(temp)
# plotWidget = pg.plot(title="Trigger raw + diff")

# plotWidget.plot(time2,temp2,pen='blue')
# plotWidget.plot(time2[0:-1],temp3,pen='white')

# # picks = mne.pick_channels_regexp(raw.info.ch_names, regexp='Xe1')
# # raw.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=False) 
# # picks = mne.pick_channels_regexp(raw.info.ch_names, regexp='')
# # raw.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=False)


#### FIND MAXIMA MINIMA

In [17]:
import numpy as np
from scipy.signal import argrelextrema

# temp3 = normalized(temp3)

# plotWidget = pg.plot(title="Trigger Max and min convolved V5")

# # for local maxima
# indexes = scipy.signal.argrelextrema(temp3, np.greater,order=200)[0]
# print(len(temp3[scipy.signal.argrelextrema(temp3, np.greater, order=200)[0][:]]))
# print(len(indexes[:]))
# plotWidget.plot(time2[0:-1],temp3,pen='white')
# plotWidget.plot(time2[0:-1],temp3,pen='white')
# plotWidget.plot(indexes,temp3[indexes],pen=None, symbol='o')

# for local minima



739
739


## ONSET TRIGGERS: 

In [18]:
# Filtered plots
# raw_original = raw.copy()

index_final_onset,_ = scipy.signal.find_peaks(temp3, height=0.8*temp3.max())
print(index_final_onset)

plotWidget = pg.plot(title="Trigger Max and min convolved V4")

# for local maxima
print(len(temp[index_final_onset]))
print(len(index_final_onset[:]))
plotWidget.plot(time2[0:-1],temp3,pen='white')
plotWidget.plot(index_final_onset,temp3[index_final_onset],pen=None, symbol='o')

[  6446   8724  11179  13876  16530  18778  21064  23520  25565  28028
  30312  32769  35226  37917  39967  42625  44875  47572  50263  52547
  54973  57050  59712  62410  64866  67319  69607  71852  74509  76757
  79452  81499  83958  86007  88499  90543  92791  94871  96919  99203
 101690 104147 106807 109471 111965 114416 116697 118742 121198 123450
 125732 128190 130646 132690 134974 137021 139072 141731 144249 146706
 149162 151412 153902 156563 159014 161297 163340 166005 168084 170333
 172586 175314 177360 179814 182269 184552 186600 189054 191094 193174
 195836 197881 199960 202033 204492 206984 209027 211280 213331 215381
 217461 219916 221997 224456 227120 229785 231863 234523 236569 238854
 241519 244180 246258 248921 251374 253832 256525 259182 261638 264334
 266788 269244 271555 273805 275850 278136 280382 282429 285128 287582
 289831 292521 294564 296849 298892 301382 303841 306501 308785 311041
 313491 315979 318233 320484 323184 325228 327888 330580 332624 335289
 33774

OFFSET TRIGGERS

In [24]:
# Filtered plots
# # raw_original = raw.copy()


# index_final_offset,_ = scipy.signal.find_peaks(-1*temp3, height=-0.2)
# print(index_final_offset)

# plotWidget = pg.plot(title="Trigger Max and min convolved V4")

# # for local maxima
# print(len(temp[index_final_offset]))
# print(len(index_final_offset[:]))
# plotWidget.plot(time2[0:-1],temp3,pen='white')
# plotWidget.plot(index_final_offset,temp3[index_final_offset],pen=None, symbol='o')
# plotWidget.plot(index_final_onset,temp3[index_final_onset],pen=None, symbol='x')

## ALL TOGETHER

In [19]:
# Filtered plots
# raw_original = raw.copy()


index_final_offset,_ = scipy.signal.find_peaks(-1*temp3, height=-0.2)
print(index_final_offset)

plotWidget = pg.plot(title="Trigger Max and min convolved V6")

# for local maxima
print(len(temp[index_final_offset]))
print(len(index_final_offset[:]))
plotWidget.plot(time2,temp2,pen='blue')
plotWidget.plot(time2[0:-1],temp3,pen='white')
plotWidget.plot(index_final_offset,temp3[index_final_offset],pen=None, symbol='o')
plotWidget.plot(index_final_onset,temp3[index_final_onset],pen=None, symbol='x')


[  7098   9588  12249  14941  17190  19440  21897  23980  26468  28691
  31177  33633  36291  38342  41037  43287  45947  48637  50922  53381
  55433  58121  60780  63270  65728  67979  70270  72921  75172  77826
  79908  82364  84416  86873  88958  91205  93251  95333  97583 100067
 102557 105216 107873 110334 112828 115078 117157 119609 121862 124113
 126598 129053 131138 133386 135400 137483 140141 142661 145115 147571
 149824 152278 154968 157426 159675 161758 164445 166463 168713 170997
 173689 175772 178225 180678 182929 184979 187468 189514 191556 194245
 196294 198342 200388 202901 205357 207408 209690 211741 213792 215877
 218327 220375 222830 225492 228224 230242 232932 234980 237231 239891
 242588 244671 247326 249783 252238 254898 257591 260046 262706 265198
 267652 269904 272218 274266 276515 278764 280842 283500 285991 288243
 290899 292945 295229 297277 299760 302249 304907 307160 309414 311907
 314357 316640 318896 321552 323641 326297 328956 331037 333695 336152
 33820

## EXPORT TEXT FILE

In [20]:
with open(os.path.join(data_path, "triggers2_PAT3066_visualCatLoc.txt"), "w") as textfile:
    textfile.write('onsets;')
    [textfile.write(str(x)+';') for x in index_final_onset]
    textfile.write('\noffsets;')
    [textfile.write(str(x)+';') for x in index_final_offset]
    textfile.close()

In [ ]:
raw.preload_data()
raw.filter(l_freq=0.15, h_freq=None, picks=picks)
raw.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=False)

raw.notch_filter(np.arange(60, 241, 60), picks=picks, filter_length='auto',phase='zero')
raw.plot_psd(area_mode='range', tmax=10.0, picks=picks, average=False)


In [22]:
# # FIND TRIGGERS FROM PHOTODIODE
# # print(len(raw.get_data().T))
# %matplotlib qt
# pd_parser.find_pd_params(raw, pd_ch_names=['Xe1'])
# pd_parser.parse_pd(raw, pd_ch_names=['Xe1'], max_len=15.83, zscore=-2.26,overwrite=True)
# # print(Xe1_params)
# # print(xe1_params2)

In [23]:
# # Some information about the DATASET: 
# print(raw.info)
# infodata = raw.info

# ## FURTHER INFORMATION
# # print(raw.info.ch_names)
# # print(raw.annotations)
# picks = mne.pick_channels_regexp(raw.info.ch_names, regexp='')
# # print(picks)
# # raw.plot(order=picks, n_channels=len(picks), duration=5, scalings=1000, start=120)
# raw2 = raw.copy()
# events = mne.find_events(raw2, stim_channel='Xe1', initial_event=False,uint_cast=True, min_duration=3/raw.info['sfreq'], )
# raw2.plot(order=picks, events=events, n_channels=203, duration=20, event_color='cyan', scalings=500, start=200)
# out_dir = _TempDir()


## FILTERING

In [25]:
# # raw2.plot(order=picks, n_channels=203, duration=20, event_color='cyan', scalings=1000, start=190)
# picks = mne.pick_channels_regexp(raw.info.ch_names, regexp='Xe1')

# raw2.plot(order=picks, n_channels=1, duration=500, event_color='cyan', scalings=5000, start=190)

# raw2.apply_hilbert(picks=picks)
# raw2.plot(order=picks, n_channels=1, duration=500, event_color='cyan', scalings=5000, start=190)


In [21]:
# FILTERING


In [ ]:

# # raw.plot(duration=60)
# picks = mne.pick_channels_regexp(raw.info.ch_names, regexp='Xe')
# raw.plot(order=picks, n_channels=len(picks), duration=1600, scalings=3000, start=120)

In [ ]:
# raw2 = raw.copy()
# raw2.info['bads'] = []
# picks = mne.pick_channels_regexp(raw2.info.ch_names, regexp='S')
# print(picks)
# raw2.plot(order=picks, n_channels=len(picks), duration=2)


In [ ]:
# raw2 = raw.copy()
# raw2.info['bads'] = []
# picks = mne.pick_channels_regexp(raw2.info.ch_names, regexp='ST')
# print(picks)
# events = mne.find_events(raw2, stim_channel='Xe1', initial_event=True,uint_cast=True)

# raw2.plot(order=picks, n_channels=len(picks), duration=5, event_color='cyan', scalings=0.1)
# print(events)
# epochs = mne.Epochs(raw2, events=events)['2'].average().plot()

### Independend Component Analysis: 
```
[CleanData, trials, BadIdx] = reject_artefact(EventData);
save TrialIndex trials
save BadChannelIndex BadIdx

popout_ica(CleanData);

[cleanBEEPData, ~ , ~] = reject_artefact(BEEPdata);

popout_ica(cleanBEEPData);
```



In [ ]:
# # set up and fit the ICA
# ica = mne.preprocessing.ICA(n_components=20, random_state=97, max_iter=800)
# ica.fit(raw)
# temporary = os.path.join(data_path, 'BadChannelIndex.mat')
# ica.exclude = scipy.io.loadmat(temporary)  # [1, 2]  # details on how we picked these are omitted here

# ica.plot_properties(raw, picks=ica.exclude)

In [ ]:
# orig_raw = raw.copy()
# raw.load_data()
# ica.apply(raw)

# # identify channels
# temporary = os.path.join(data_path, 'Params.mat')
# params = scipy.io.loadmat(temporary) 

# # show some frontal channels to clearly illustrate the artifact removal
# chs =  params.channels # ['e1','e2']
# chan_idxs = [raw.ch_names.index(ch) for ch in chs]
# orig_raw.plot(order=chan_idxs, start=12, duration=4)
# raw.plot(order=chan_idxs, start=12, duration=4)

### Triggers: 
```
events = mne.find_events(raw, stim_channel='STI 014')
print(events[:5])  # show the first 5
```

In [ ]:
# events = mne.find_events(raw, stim_channel='STI 014')
# print(events[:5])  # show the first 5

# # event_dict = {'auditory/left': 1, 'auditory/right': 2, 'visual/left': 3,
# #               'visual/right': 4, 'smiley': 5, 'buttonpress': 32} 
# event_dict = params.triggers

# fig = mne.viz.plot_events(events, event_id=event_dict, sfreq=raw.info['sfreq'],
#                           first_samp=raw.first_samp)